In [141]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs
from utils import averageModels, normalModels

In [142]:
class Arguments():
    def __init__(self):
        self.images = 60000
        self.clients = 20
        self.rounds = 10
        self.epochs = 5
        self.local_batches = 64
        self.lr = 0.01
        self.C = 1.0
        self.drop_rate = 0.0
        self.torch_seed = 0
        self.log_interval = 100
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = False
        self.save_model = False

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [143]:
writer = open('20_experiments_fashio.csv', 'a')
writer.write('\n\n\n')
writer.write('Clients {}, Rounds {}, Epochs {}, IID {}'.format(args.clients, args.rounds, args.epochs, args.iid))
writer.write('\n')

1

In [144]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

In [145]:
# # Download MNIST manually using 'wget' then uncompress the file
# !wget www.di.ens.fr/~lelarge/MNIST.tar.gz
# !tar -zxvf MNIST.tar.gz

In [146]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

In [147]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
#     client['samples'] = len(trainset_ind_list) / args.images
    client['samples'] = len(trainset_ind_list) / (len(trainset_ind_list) * args.clients)

In [148]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = datasets.FashionMNIST('./', train=False, download=True, transform=transform)
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

In [149]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [150]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
#             data = data.send(client['hook'])
#             target = target.send(client['hook'])
            
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = F.nll_loss(output, target)
            loss.backward()
            client['optim'].step()
            
            if batch_idx % args.log_interval == 0:
#                 loss = loss.get() 
                print('Model {} Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, batch_idx * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * batch_idx / len(client['trainset']), loss))
                
#     client['model'].get() 

In [151]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    writer.write('{},'.format(100. * correct / len(test_loader.dataset)))

In [152]:
def Aggregate(all_models, sample_client):
    all_dicts = {}
    for inds, client in enumerate(all_models):
        all_dicts[client['hook'].id] = all_models[inds]['model'].state_dict()
    output_dict = copy.deepcopy(all_dicts[sample_client])
    for key in all_dicts[sample_client]:
        for val in all_dicts:
            if val == sample_client:
                continue
            output_dict[key] += all_dicts[val][key]
#         output_dict[key] /= float(len(all_dicts))
        output_dict[key] /= float(len(all_models))
    output_model = Net().to(device)
    output_model.load_state_dict(output_dict)
    return output_model

In [153]:
torch.manual_seed(args.torch_seed)
global_model = Net()
# torch.manual_seed(args.torch_seed)
# normal_avg = Net()
# torch.manual_seed(args.torch_seed)
# fake_model = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)

start_time = time.time()

for fed_round in range(args.rounds):
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    
    # Training 
    for client in active_clients:
        ClientUpdate(args, device, client)
    
#     # Testing 
#     for client in active_clients:
#         test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
#     normal_avg = normalModels(normal_avg, active_clients)
#     fake_model = Aggregate(active_clients, 'client1')
    
    # Testing the average model
    test(args, global_model, device, global_test_loader, 'Global')
#     test(args, normal_avg, device, global_test_loader, 'Global')
#     test(args, fake_model, device, global_test_loader, 'Global')
    
#     break
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())

writer.write('\n')
end_time = time.time() 
hours, rem = divmod(end_time-start_time, 3600)
minutes, seconds = divmod(rem, 60)
writer.write('{:0>2}:{:0>2}:{:05.2f}'.format(int(hours),int(minutes),seconds))
writer.write('\n')
writer.close()

if (args.save_model):
    torch.save(global_model.state_dict(), "FedAvg.pt")

Model client16 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.302624
Model client16 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.050586
Model client16 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.280409
Model client16 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.091421
Model client16 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.982444
Model client2 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.317857
Model client2 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.121145
Model client2 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.386963
Model client2 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.287514
Model client2 Train Epoch: 5 [0/3008 (0%)]	Loss: 1.132966
Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.316322
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.045229
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 1.516021
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 1.245880
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.724218
Model client5 Train Epoch: 1 [0/3008 (0%)]	Loss: 2.312681
Model client5 Train Epoch: 2 [0/3008 (0%)]	Loss: 2.033007
Mode

Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.796956
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.710070
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.561995
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.742975
Model client18 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.728493
Model client18 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.681000
Model client17 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.918503
Model client17 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.773849
Model client17 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.614874
Model client17 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.732485
Model client17 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.799731
Model client19 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.819061
Model client19 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.931473
Model client19 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.860278
Model client19 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.755014
Model client19 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.713695
Model client4 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.85625

Model client4 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.703946
Model client4 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.781111
Model client9 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.685908
Model client9 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.479169
Model client9 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.652120
Model client9 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.662068
Model client9 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.695956
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.841112
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.570486
Model client3 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.872209
Model client3 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.733426
Model client3 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.595196
Model client7 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.620833
Model client7 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.987293
Model client7 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.609584
Model client7 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.558577
Model client7 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.777488
Model client2 

Model client1 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.670342
Model client1 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.700229
Model client1 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.579810
Model client1 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.352058
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.502928
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.828887
Model client3 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.400221
Model client3 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.557704
Model client3 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.581463
Model client14 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.635852
Model client14 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.740775
Model client14 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.656014
Model client14 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.487701
Model client14 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.682980
Model client20 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.577834
Model client20 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.575055
Model client20 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.391387
Model 

Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.564450
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.637716
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.443345
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.612436
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.416829
Model client14 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.729511
Model client14 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.430000
Model client14 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.587149
Model client14 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.712254
Model client14 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.490621
Model client19 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.375938
Model client19 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.620837
Model client19 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.441574
Model client19 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.491928
Model client19 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.317450
Model client3 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.555210
Model client3 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.454131

Model client12 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.579645
Model client13 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.414296
Model client13 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.575372
Model client13 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.421272
Model client13 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.547467
Model client13 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.580320

Test set: Average loss for Global model: 0.5106, Accuracy: 8175/10000 (82%)

Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.355191
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.434224
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.507314
Model client18 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.401371
Model client18 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.411818
Model client4 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.475443
Model client4 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.322039
Model client4 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.416125
Model client4 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.439862
Model client4 Train Epoch: 5 [0/3008 (0%)

Model client15 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.417809
Model client15 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.410939
Model client15 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.477878
Model client7 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.497401
Model client7 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.307386
Model client7 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.448515
Model client7 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.582705
Model client7 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.409150
Model client20 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.452388
Model client20 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.413919
Model client20 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.390314
Model client20 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.415435
Model client20 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.451346
Model client12 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.366224
Model client12 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.327341
Model client12 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.446916
Model client12 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.389416
Mo

Model client15 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.491383
Model client15 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.391089
Model client15 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.437565
Model client15 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.320849
Model client1 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.460075
Model client1 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.498212
Model client1 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.612944
Model client1 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.538783
Model client1 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.485893
Model client2 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.426845
Model client2 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.435050
Model client2 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.355863
Model client2 Train Epoch: 4 [0/3008 (0%)]	Loss: 0.526183
Model client2 Train Epoch: 5 [0/3008 (0%)]	Loss: 0.550551
Model client18 Train Epoch: 1 [0/3008 (0%)]	Loss: 0.582923
Model client18 Train Epoch: 2 [0/3008 (0%)]	Loss: 0.421953
Model client18 Train Epoch: 3 [0/3008 (0%)]	Loss: 0.439876
Model c

In [154]:
clients[9]['trainset']